In [2]:
import numpy as np
import pandas as pd

In [3]:
np.__version__

'1.14.2'

In [244]:
class Tensor:
    def __init__(self,array,grad=None,requires_grad=False,name='w1'):
        self.array=array
        self.__requires_grad=requires_grad
         
        if requires_grad:
            #name of the parameter
            self.grad={name: self.make_grad()}
        else:
            self.grad={name: 0}
        if grad is not None:
            self.grad=grad

    def make_grad(self,):
        shape=self.array.shape
        Kron=1
        for d in shape:
            ID=np.identity(d)
            Kron=np.tensordot(Kron,ID,axes=0)
        new_shape=[i for i in range(0,2*len(shape),2)]
        new_shape+=[i for i in range(1,2*len(shape),2)]
        Kron=Kron.transpose(new_shape)

        return Kron
    @property
    def shape(self):
        return self.array.shape
    @property
    def ndim(self):
        return self.array.ndim
    @property
    def grad_shape(self):
        return self.grad.shape

    def __add__(self,x):
        result=self.array+x.array

        grad={}
        for w in self.grad:
            if w not in x.grad:
                x.grad[w]=0
            else:
                grad[w]=self.grad[w]+x.grad[w]
        for w in x.grad:
            if w not in self.grad:
                self.grad[w]=0

        return Tensor(result,grad=grad)

    def __mul__(self,x):
        if isinstance(x,int) or isinstance(x,float):
            result=x*self.array
            grad=x*self.grad

        if isinstance(x,Tensor):
            result=np.tensordot(self.array,x.array,axes=([-1],[0]))

            for w in self.grad:
                if w not in x.grad:
                    x.grad[w]=0
            for w in x.grad:
                if w not in self.grad:
                    self.grad[w]=0

            grad={}
            for w in self.grad:
                if x.grad[w] is 0:
                    grad1=0
                else:
                    grad1=np.tensordot(self.array,x.grad[w],axes=([-1],[0]))
                    
                if self.grad[w] is 0:
                    grad2=0
                else:
                    i=len(self.array.shape)
                    grad2=np.tensordot(self.grad[w],x.array,axes=([i-1],[0]))
                    n1=self.grad[w].ndim
                    n2=self.array.ndim
                    n3=x.array.ndim
                    r1=[j for j in range(n2-1)]+[j for j in range(n1-1,n1+n3-2)]
                    r2=[j for j in range(n2-1,n1-1)]
                    grad2=grad2.transpose(r1+r2)
                
                grad[w]=grad1+grad2

        return Tensor(result,grad=grad)
    
    def __rmul__(self, x):
        if isinstance(x,int) or isinstance(x,float):
            result=x*self.array
            grad=x*self.grad
        
        return Tensor(result,grad=grad)

    def sum(self,axis):
        result=self.array.sum(axis=axis)
        grad={}
        for w in self.grad:
            grad[w]=self.grad[w].sum(axis=axis)
        return Tensor(result,grad=grad)

    def __repr__(self):

        return f'Tensor({self.array},dtype {self.array.dtype},requires_grad={self.__requires_grad})'

In [271]:
class Sigmoid:
    """
    returns: Tensor with gradients
    """
    def __call__(self,x):

        out=1/(1+np.exp(-x.array))
        grad={}
        for w in x.grad:
            grad[w]=self.grad(x)*x.grad[w]

        return Tensor(out,grad=grad) 

    def grad(self,x):
        den=(1+np.exp(-x.array))*(1+np.exp(-x.array))
        gd=np.exp(-x.array)/den

        return gd

class Exp:
    """
    returns: Tensor with gradients
    """
    def __call__(self,x):

        out=np.exp(x.array)
        grad=self.grad(x)*x.grad

        return Tensor(out,grad=grad) 
    
    def grad(self,x):
        gd=np.exp(x.array)
        return gd


In [262]:
x=np.random.rand(3,5)
y=np.random.rand(5,1)

In [266]:
X=Tensor(x,requires_grad=True,name='w1')
Y=Tensor(y,requires_grad=True,name='w2')

In [269]:
z=np.random.rand(1,3)
Z=Tensor(z)

In [272]:
sig=Sigmoid()

In [273]:
out=Z*X
out=sig(out)
out=out*Y

In [274]:
out.shape

(1, 1)

In [277]:
out.grad['w1'].shape

(1, 1, 3, 5)